<a href="https://colab.research.google.com/github/racoope70/exploratory_daytrading/blob/main/multi_stock_sac_inference_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Fix Potential Library Conflicts
!apt-get remove --purge -y cuda* libcuda* nvidia* || echo "No conflicting CUDA packages"
!apt-get autoremove -y
!apt-get clean

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'cuda-toolkit-12-4-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-7' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-8' for glob 'cuda*'
Note, selecting 'cuda-toolkit-12-5-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-2' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-3' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-4' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-5' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-6' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-8' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-9' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-13-0' fo

In [2]:
#Protocol Buffer Fix (for TensorFlow)
!pip uninstall -y protobuf
!pip install protobuf==3.20.3

Found existing installation: protobuf 5.29.5
Uninstalling protobuf-5.29.5:
  Successfully uninstalled protobuf-5.29.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 11.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.


In [3]:
#Update Colab Environment and System Libraries
!apt-get update -y && apt-get upgrade -y


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:6 https://cli.github.com/packages stable InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,623 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,579 kB]
Ge

In [4]:
#Install Correct Version of CUDA for Colab GPU
!apt-get update -qq && apt-get install -y \
    libcusolver11 libcusparse11 libcurand10 libcufft10 libnppig10 libnppc10 libnppial10 \
    cuda-toolkit-12-4

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libnppig10
E: Unable to locate package libnppc10
E: Unable to locate package libnppial10


In [5]:
#Set Correct CUDA Paths
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda-12.4'
os.environ['PATH'] += ':/usr/local/cuda-12.4/bin'
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda-12.4/lib64'


In [6]:
#Install RAPIDS and NVIDIA Dependencies
!pip install --extra-index-url=https://pypi.nvidia.com \
    cuml-cu12==25.2.0 cudf-cu12==25.2.0 cupy-cuda12x dask-cuda==25.2.0 dask-cudf-cu12==25.2.0


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.0/405.0 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.2/851.2 kB 160.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.5/293.5 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 168.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.7/557.7 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.2/27.2 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [1]:
#Install TensorFlow (latest GPU-compatible version)
!pip install tensorflow==2.18.0

#Install Stable Baselines3 and Trading Libraries
!pip install stable-baselines3[extra] gymnasium gym-anytrading yfinance xgboost joblib

#Install Miscellaneous Libraries
!pip install matplotlib scikit-learn pandas numba==0.61.0

#Install PyTorch with GPU Support
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


  Using cached tensorflow-2.18.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
Using cached tensorflow-2.18.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (615.5 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tensorflow 2.18.0 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.18.0 which is incompatible.
tf-keras 2.19.0 requires tensorflow<2.20,>=2.19, but you have tensorflow 2.18.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#Install TensorFlow (latest GPU-compatible version)
!pip install tensorflow==2.18.0

#Immediately follow it with:
import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("TensorFlow GPU memory growth enabled")
    except RuntimeError as e:
        print(f"TensorFlow GPU memory config failed: {e}")


In [3]:
!rm -rf /content/drive

In [4]:
import os, gc, time, logging, sys, re
from datetime import datetime
import numpy as np
import pandas as pd
import yfinance as yf
import pywt

# ---------------- Paths ----------------
BASE_DIR   = "./sac_parity"
DATA_DIR   = os.path.join(BASE_DIR, "data")
LOG_DIR    = os.path.join(BASE_DIR, "logs")
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

# Try to use Google Drive in Colab; otherwise use local CWD
try:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive', force_remount=False)
    DRIVE_BASE = "/content/drive/MyDrive"
except Exception:
    DRIVE_BASE = os.getcwd()

RESULTS_DIR = os.path.join(DRIVE_BASE, "Results_May_2025", "results_sac_walkforward")
os.makedirs(RESULTS_DIR, exist_ok=True)
FEATURE_CSV = os.path.join(RESULTS_DIR, "multi_stock_feature_engineered_dataset.csv")

# ---------------- Logger ----------------
def setup_logger(name="download_fe", level=logging.INFO):
    logger = logging.getLogger(name)
    logger.setLevel(level)
    # Avoid duplicate handlers if this script/cell runs more than once
    logger.handlers = []
    fmt = logging.Formatter("%(asctime)s | %(levelname)s | %(name)s | %(message)s")

    sh = logging.StreamHandler(sys.stdout)
    sh.setFormatter(fmt); sh.setLevel(level)
    logger.addHandler(sh)

    fh = logging.FileHandler(os.path.join(LOG_DIR, f"{name}.log"))
    fh.setFormatter(fmt); fh.setLevel(level)
    logger.addHandler(fh)
    return logger

log = setup_logger("download_fe", level=logging.INFO)

# ---------------- Feature Toggles ----------------
USE_SENTIMENT  = False   # placeholder (constant 0.0 if False)
USE_REGIME     = True
USE_WAVELET    = True
USE_GREEKS     = False   # not used in this script; kept for parity

# ---------------- Data Params ----------------
INTERVAL    = "1h"   # yfinance interval
PERIOD_DAYS = 720    # how many days of history to pull

# ---------------- Universe ----------------
TICKERS = [
    'AAPL','TSLA','MSFT','GOOGL','AMZN','NVDA','META','BRK-B','JPM','JNJ',
    'XOM','V','PG','UNH','MA','HD','LLY','MRK','PEP','KO',
    'BAC','ABBV','AVGO','PFE','COST','CSCO','TMO','ABT','ACN','WMT',
    'MCD','ADBE','DHR','CRM','NKE','INTC','QCOM','NEE','AMD','TXN',
    'AMGN','UPS','LIN','PM','UNP','BMY','LOW','RTX','CVX','IBM',
    'GE','SBUX','ORCL'
]
SYMBOLS = TICKERS  # override elsewhere if needed (e.g., short tests)

# ---------------- Helpers: schema + datetime ----------------
def _force_datetime_column(df: pd.DataFrame) -> pd.DataFrame:
    """Ensure tz-naive 'Datetime' exists, deduplicate and sort by Datetime."""
    if isinstance(df.index, pd.DatetimeIndex):
        # Remove timezone to keep consistent CSV/Parquet
        try:
            df.index = df.index.tz_convert(None)
        except Exception:
            try:
                df.index = df.index.tz_localize(None)
            except Exception:
                pass
        df.index.name = 'Datetime'
        df = df.reset_index()
    else:
        df = df.reset_index()
        first = df.columns[0]
        if np.issubdtype(df[first].dtype, np.datetime64):
            df = df.rename(columns={first: 'Datetime'})
        elif 'Date' in df.columns:
            df['Datetime'] = pd.to_datetime(df['Date'])
        elif 'Datetime' not in df.columns:
            df['Datetime'] = pd.to_datetime(df[first], errors='coerce')

    if 'Datetime' not in df.columns:
        raise KeyError("Failed to construct 'Datetime' from data.")
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    return df.drop_duplicates(subset=['Datetime']).sort_values('Datetime').reset_index(drop=True)

def _normalize_ohlcv(df_in: pd.DataFrame, ticker: str) -> pd.DataFrame:
    """Flatten MultiIndex, strip ticker prefixes/suffixes, map to canonical OHLCV names."""
    df = df_in.copy()
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(p) for p in col if p]) for col in df.columns]
    df.columns = [re.sub(r"\s+", " ", str(c)).strip() for c in df.columns]

    tkr = ticker.upper().replace("-", "[- ]?")
    cleaned = {}
    for c in df.columns:
        cu = c.upper()
        cu = re.sub(rf"^(?:{tkr})[\s/_-]+", "", cu)
        cu = re.sub(rf"[\s/_-]+(?:{tkr})$", "", cu)
        cleaned[c] = cu.title()
    if any(cleaned[c] != c for c in df.columns):
        df = df.rename(columns=cleaned)

    cols_ci = {c.lower(): c for c in df.columns}
    wants = {
        "Open":      ["open"],
        "High":      ["high"],
        "Low":       ["low"],
        "Close":     ["close","last","close*"],
        "Adj Close": ["adj close","adj_close","adjclose","adjusted close"],
        "Volume":    ["volume","vol"]
    }
    rename_map = {}
    for desired, alts in wants.items():
        if desired.lower() in cols_ci:
            rename_map[cols_ci[desired.lower()]] = desired
        else:
            for a in alts:
                if a in cols_ci:
                    rename_map[cols_ci[a]] = desired
                    break
    if rename_map:
        df = df.rename(columns=rename_map)
    return df

# ---------------- Downloader (retry + history() fallback) ----------------
def download_stock_data(ticker, interval="1h", period_days=720, max_retries=5, sleep_base=3):
    """
    Robust yfinance intraday downloader.
    Guarantees: Open, High, Low, Close, Volume (+Adj Close), Datetime, Symbol.
    """
    period_str = f"{int(period_days)}d"

    def _post(df: pd.DataFrame) -> pd.DataFrame:
        df = _normalize_ohlcv(df, ticker)
        df = _force_datetime_column(df)
        needed = {'Open', 'High', 'Low', 'Close', 'Volume'}
        missing = needed - set(df.columns)
        if missing:
            raise ValueError(f"Missing OHLCV columns after normalize: {missing}")
        if 'Adj Close' not in df.columns:
            df['Adj Close'] = df['Close']
        return df

    for attempt in range(1, max_retries + 1):
        try:
            log.info(f"[{ticker}] Attempt {attempt}: download(period={period_str}, interval={interval})")
            df = yf.download(
                tickers=ticker,
                period=period_str,
                interval=interval,
                progress=False,
                auto_adjust=False,
                group_by='column',
                threads=False,
                prepost=False,
                repair=True
            )
            if df is None or df.empty:
                raise ValueError("Empty data from download()")
            df = _post(df)
            df['Symbol'] = ticker
            log.info(f"[{ticker}] rows={len(df)} {df['Datetime'].min()} → {df['Datetime'].max()}")
            return df

        except Exception as e1:
            log.warning(f"[{ticker}] download() error: {e1} | trying Ticker().history()")
            try:
                hist = yf.Ticker(ticker).history(
                    period=period_str,
                    interval=interval,
                    auto_adjust=False,
                    actions=False
                )
                if hist is None or hist.empty:
                    raise ValueError("Empty data from history()")
                df = _post(hist)
                df['Symbol'] = ticker
                log.info(f"[{ticker}] (fallback) rows={len(df)} {df['Datetime'].min()} → {df['Datetime'].max()}")
                return df
            except Exception as e2:
                wait = sleep_base * attempt
                log.warning(f"[{ticker}] history() error: {e2} | retrying in {wait}s")
                time.sleep(wait)

    log.error(f"[{ticker}] Failed after {max_retries} attempts.")
    return None

# ---------------- Feature Engineering (parity) ----------------
def denoise_wavelet(series, wavelet='db1', level=2):
    s = pd.Series(series).astype(float).ffill().bfill().to_numpy()
    try:
        coeffs = pywt.wavedec(s, wavelet, mode='symmetric', level=level)
        # hard smoothing: zero high-frequency detail
        for i in range(1, len(coeffs)):
            coeffs[i] = np.zeros_like(coeffs[i])
        rec = pywt.waverec(coeffs, wavelet, mode='symmetric')
        return pd.Series(rec[:len(s)], index=series.index)
    except Exception as e:
        log.warning(f"Wavelet denoise failed ({e}); returning raw series.")
        return pd.Series(s, index=series.index)

def add_regime(df: pd.DataFrame) -> pd.DataFrame:
    df['Vol20'] = df['Close'].pct_change().rolling(20).std()
    df['Ret20'] = df['Close'].pct_change(20)
    vol_hi   = (df['Vol20'] > df['Vol20'].median()).astype(int)
    trend_hi = (df['Ret20'].abs() > df['Ret20'].abs().median()).astype(int)
    df['Regime4'] = vol_hi * 2 + trend_hi  # 0..3
    return df

def compute_enhanced_features(df: pd.DataFrame) -> tuple[pd.DataFrame, list]:
    """
    Returns (feature_df, FEATURES) with PPO-parity indicators for SAC.
    Input df must contain: Datetime, Symbol, Open, High, Low, Close, Volume.
    """
    df = df.copy()
    req = {"Open", "High", "Low", "Close", "Volume"}
    assert req.issubset(df.columns), f"OHLCV columns missing: {req - set(df.columns)}"

    # Work in float64 for stability, cast to float32 at the end
    close = df["Close"].astype("float64")
    high  = df["High"].astype("float64")
    low   = df["Low"].astype("float64")
    open_ = df["Open"].astype("float64")
    vol   = (df["Volume"].astype("float64") + 1.0)  # avoid div-by-zero

    # --- Returns & lags ---
    df["ret_1"]     = close.pct_change(1)
    df["ret_3"]     = close.pct_change(3)
    df["ret_5"]     = close.pct_change(5)
    df["ret_10"]    = close.pct_change(10)
    df["logret_1"]  = np.log(close).diff(1)

    # --- Moving avgs / vol ---
    df["ma_5"]      = close.rolling(5).mean()
    df["ma_10"]     = close.rolling(10).mean()
    df["ma_20"]     = close.rolling(20).mean()
    df["ema_10"]    = close.ewm(span=10, adjust=False).mean()
    df["ema_20"]    = close.ewm(span=20, adjust=False).mean()
    df["std_10"]    = close.rolling(10).std()
    df["std_20"]    = close.rolling(20).std()

    # Normalize some to price
    df["ema10_ratio"] = df["ema_10"] / close - 1.0
    df["ema20_ratio"] = df["ema_20"] / close - 1.0

    # --- Z-score of price ---
    df["z_close_20"] = (close - df["ma_20"]) / df["std_20"].replace(0, np.nan)

    # --- RSI(14) ---
    delta      = close.diff()
    up         = delta.clip(lower=0.0)
    down       = -delta.clip(upper=0.0)
    roll_up    = up.ewm(alpha=1/14, adjust=False).mean()
    roll_down  = down.ewm(alpha=1/14, adjust=False).mean()
    rs         = roll_up / roll_down.replace(0, np.nan)
    df["rsi_14"] = 100 - (100 / (1 + rs))

    # --- Stochastic(14,3) with guarded denominator ---
    ll14       = low.rolling(14).min()
    hh14       = high.rolling(14).max()
    den_14     = (hh14 - ll14).replace(0, np.nan)
    df["stoch_k"] = 100 * (close - ll14) / den_14
    df["stoch_d"] = df["stoch_k"].rolling(3).mean()

    # --- MACD (12,26,9) ---
    ema12         = close.ewm(span=12, adjust=False).mean()
    ema26         = close.ewm(span=26, adjust=False).mean()
    macd          = ema12 - ema26
    macd_sig      = macd.ewm(span=9, adjust=False).mean()
    df["macd"]        = macd
    df["macd_signal"] = macd_sig
    df["macd_hist"]   = macd - macd_sig

    # --- Bollinger Bands (20,2) with guards ---
    bb_mid      = df["ma_20"]
    bb_std      = df["std_20"]
    bb_up       = bb_mid + 2 * bb_std
    bb_lo       = bb_mid - 2 * bb_std
    band_width  = (bb_up - bb_lo)
    df["bb_perc_b"]   = (close - bb_lo) / band_width.replace(0, np.nan)
    df["bb_bandwidth"] = band_width / bb_mid.replace(0, np.nan)

    # --- ATR(14) (rolling mean of True Range) ---
    prev_close  = close.shift(1)
    tr = np.maximum(high - low,
                    np.maximum((high - prev_close).abs(), (low - prev_close).abs()))
    df["atr_14"] = tr.rolling(14).mean()

    # --- Volume features ---
    df["vol_ma_20"]    = vol.rolling(20).mean()
    df["vol_std_20"]   = vol.rolling(20).std()
    df["vol_z_20"]     = (vol - df["vol_ma_20"]) / df["vol_std_20"].replace(0, np.nan)
    df["vol_change_1"] = vol.pct_change(1)

    # --- Spreads / micro-structure-ish ---
    df["hl_spread"] = (high - low) / close.replace(0, np.nan)
    df["oc_spread"] = (close - open_) / open_.replace(0, np.nan)

    # --- Simple cross signal (numeric) ---
    df["ema10_gt_ema20"] = (df["ema_10"] > df["ema_20"]).astype("float32")

    # Optional extras
    if USE_WAVELET:
        df["denoised_close"] = denoise_wavelet(pd.Series(close, index=df.index))
    if USE_REGIME:
        df = add_regime(df)  # adds 'Regime4'

    # Replace infs, then fill remaining NaNs from rolling warm-ups
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(0.0, inplace=True)

    # Explicit feature list (order matters)
    FEATURES = [
        "ret_1","ret_3","ret_5","ret_10","logret_1",
        "ma_5","ma_10","ma_20","ema_10","ema_20","ema10_ratio","ema20_ratio",
        "std_10","std_20","z_close_20",
        "rsi_14","stoch_k","stoch_d",
        "macd","macd_signal","macd_hist",
        "bb_perc_b","bb_bandwidth",
        "atr_14",
        "vol_ma_20","vol_std_20","vol_z_20","vol_change_1",
        "hl_spread","oc_spread","ema10_gt_ema20",
    ]

    # Ensure feature dtypes are float32 for speed
    df[FEATURES] = df[FEATURES].astype("float32")

    return df, FEATURES

# ---------------- Orchestrator ----------------
def build_features_if_needed() -> pd.DataFrame:
    """
    If FEATURE_CSV exists, load and return it (no download).
    Else:
      - download each SYMBOL,
      - compute enhanced features,
      - save per-symbol Parquet to DATA_DIR,
      - write combined CSV to FEATURE_CSV and also a local copy,
      - return combined DataFrame.
    """
    if os.path.exists(FEATURE_CSV):
        log.info(f"Using existing features CSV: {FEATURE_CSV}")
        d = pd.read_csv(FEATURE_CSV)
        d['Datetime'] = pd.to_datetime(d['Datetime'])
        return d

    all_dfs = []
    for i, ticker in enumerate(SYMBOLS, 1):
        log.info(f"[{i}/{len(SYMBOLS)}] {ticker} — downloading")
        raw = download_stock_data(ticker, interval=INTERVAL, period_days=PERIOD_DAYS)
        if raw is None or raw.empty:
            log.warning(f"[{ticker}] no data; skipping.")
            continue
        try:
            feats, _ = compute_enhanced_features(raw)
            if feats is not None and not feats.empty:
                all_dfs.append(feats)
                out_parq = os.path.join(DATA_DIR, f"{ticker}.parquet")
                feats.to_parquet(out_parq, index=False)
                log.info(f"[{ticker}] features={len(feats)} rows → {out_parq}")
            else:
                log.warning(f"[{ticker}] empty features; skipped.")
        except Exception as e:
            log.error(f"[{ticker}] FE failed: {e}")
        finally:
            del raw
            try: del feats
            except: pass
            gc.collect()
            time.sleep(0.2)

    if not all_dfs:
        raise RuntimeError("No usable data found for any ticker.")

    full = pd.concat(all_dfs, ignore_index=True)
    # Save to Drive path and a local mirror
    local_out = "multi_stock_feature_engineered_dataset.csv"
    full.to_csv(FEATURE_CSV, index=False)  # Drive/mounted path
    full.to_csv(local_out, index=False)    # local copy for training

    # Verify artifacts (logger only; no prints)
    for p in (local_out, FEATURE_CSV):
        exists = os.path.exists(p)
        if not exists:
            log.error("Artifact missing: %s", p)
            continue
        sz = os.path.getsize(p)
        tm = time.ctime(os.path.getmtime(p))
        try:
            head = pd.read_csv(p, nrows=2)
            cols = list(head.columns)
            sample = head.to_string(index=False)
        except Exception as e:
            cols, sample = [], f"<read error: {e}>"
        log.info("Artifact: %s | exists=%s | size=%s bytes | modified=%s | cols=%s",
                 p, exists, f"{sz:,}", tm, cols[:8])
        log.info("Head preview:\n%s", sample)

    log.info("Saved combined CSV → %s (rows=%d)", FEATURE_CSV, len(full))
    return full

# --------------- Runner / Diagnostics ---------------
def _probe(path, nrows=2):
    ok = os.path.exists(path)
    log.info("Check: %s | exists=%s", path, ok)
    if not ok:
        return
    sz = os.path.getsize(path)
    tm = time.ctime(os.path.getmtime(path))
    log.info("  size=%s bytes | modified=%s", f"{sz:,}", tm)
    try:
        head = pd.read_csv(path, nrows=nrows)
        log.info("  cols: %s", list(head.columns)[:10])
        log.info("\n%s", head.to_string(index=False))
    except Exception as e:
        log.warning("  head() failed: %s", e)

if __name__ == "__main__":
    # Ensure logs are visible
    log.setLevel(logging.INFO)
    for h in log.handlers:
        h.setLevel(logging.INFO)

    log.info("=== START: download + feature build ===")
    try:
        df_built = build_features_if_needed()   # triggers download + FE
        log.info("Built features shape: %s", getattr(df_built, "shape", None))
    except Exception as e:
        log.exception("Build failed: %s", e)

    # Prove both CSVs (Drive mirror + local)
    _probe(FEATURE_CSV)
    _probe("multi_stock_feature_engineered_dataset.csv")
    log.info("Logs also written to: %s", os.path.join(LOG_DIR, "download_fe.log"))


2025-09-03 22:56:45,552 | INFO | download_fe | === START: download + feature build ===


INFO:download_fe:=== START: download + feature build ===


2025-09-03 22:56:45,559 | INFO | download_fe | [1/53] AAPL — downloading


INFO:download_fe:[1/53] AAPL — downloading


2025-09-03 22:56:45,562 | INFO | download_fe | [AAPL] Attempt 1: download(period=720d, interval=1h)


...

INFO:download_fe:  cols: ['Datetime', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Repaired?', 'Volume', 'Symbol', 'ret_1']


2025-09-03 22:58:28,884 | INFO | download_fe | 
           Datetime  Adj Close      Close       High        Low       Open  Repaired?   Volume Symbol     ret_1  ret_3  ret_5  ret_10  logret_1  ma_5  ma_10  ma_20    ema_10   ema_20  std_10  std_20  ema10_ratio  ema20_ratio  z_close_20  rsi_14  stoch_k  stoch_d      macd  macd_signal  macd_hist  bb_perc_b  bb_bandwidth  atr_14  vol_ma_20  vol_std_20  vol_z_20  vol_change_1  hl_spread  oc_spread  ema10_gt_ema20  denoised_close  Vol20  Ret20  Regime4
2022-10-19 13:30:00 142.550716 144.800400 144.949203 141.500000 141.690002       True 16147243   AAPL  0.000000    0.0    0.0     0.0  0.000000   0.0    0.0    0.0 144.80040 144.8004     0.0     0.0     0.000000     0.000000         0.0     0.0      0.0      0.0  0.000000     0.000000   0.000000        0.0           0.0     0.0        0.0         0.0       0.0      0.000000   0.023820   0.021952             0.0      143.616348    0.0    0.0        0
2022-10-19 14:30:00 141.280359 143.509995 14

INFO:download_fe:
           Datetime  Adj Close      Close       High        Low       Open  Repaired?   Volume Symbol     ret_1  ret_3  ret_5  ret_10  logret_1  ma_5  ma_10  ma_20    ema_10   ema_20  std_10  std_20  ema10_ratio  ema20_ratio  z_close_20  rsi_14  stoch_k  stoch_d      macd  macd_signal  macd_hist  bb_perc_b  bb_bandwidth  atr_14  vol_ma_20  vol_std_20  vol_z_20  vol_change_1  hl_spread  oc_spread  ema10_gt_ema20  denoised_close  Vol20  Ret20  Regime4
2022-10-19 13:30:00 142.550716 144.800400 144.949203 141.500000 141.690002       True 16147243   AAPL  0.000000    0.0    0.0     0.0  0.000000   0.0    0.0    0.0 144.80040 144.8004     0.0     0.0     0.000000     0.000000         0.0     0.0      0.0      0.0  0.000000     0.000000   0.000000        0.0           0.0     0.0        0.0         0.0       0.0      0.000000   0.023820   0.021952             0.0      143.616348    0.0    0.0        0
2022-10-19 14:30:00 141.280359 143.509995 144.899994 143.274994 144.809998

2025-09-03 22:58:28,888 | INFO | download_fe | Logs also written to: ./sac_parity/logs/download_fe.log


INFO:download_fe:Logs also written to: ./sac_parity/logs/download_fe.log


In [ ]:
# --- strict determinism block (very top) ---
import os, random
os.environ["PYTHONHASHSEED"] = "42"
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"

import numpy as np
import torch

try:
    torch.set_num_interop_threads(1)
except RuntimeError:
    pass
try:
    torch.set_num_threads(1)
except RuntimeError:
    pass

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cuda.matmul.allow_tf32 = False
    torch.backends.cudnn.allow_tf32 = False
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
try:
    torch.use_deterministic_algorithms(True)
except Exception:
    pass

# ---- libs used below (import after the determinism block) ----
import gc, time, joblib, json, re, sys, logging, warnings, pywt
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta, timezone
import heapq

# gymnasium + SB3
import gymnasium as gym
from gymnasium.spaces import Box
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnNoModelImprovement
from stable_baselines3.common.utils import set_random_seed  # <-- add

# sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

warnings.filterwarnings("ignore", message=".*Gym has been unmaintained.*")
warnings.filterwarnings("ignore", category=DeprecationWarning, module="jupyter_client.session")

# Seed SB3 utilities (matches PPO pattern)
set_random_seed(SEED)

TOP_N_WINDOWS = 3

# ========= Logging =========
TEST_MODE = True
FAST_TEST = True  # define first so it can influence LOG_LEVEL
LOG_LEVEL = os.getenv("LOG_LEVEL", "INFO").upper()
if FAST_TEST and LOG_LEVEL == "INFO":
    LOG_LEVEL = "WARNING"  # quiet during fast smoke tests

logging.basicConfig(
    level=getattr(logging, LOG_LEVEL, logging.INFO),
    format=":%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler(sys.stdout), logging.FileHandler("sac_run.log", mode="a")],
)
log = logging.getLogger("SAC-Resume")

# Where to save results (Colab-style path by default; override via env)
DRIVE_BASE = os.getenv("DRIVE_BASE", "/content/drive/MyDrive")
os.makedirs(DRIVE_BASE, exist_ok=True)

BASE_DIR   = "./sac_parity"
DATA_DIR   = os.path.join(BASE_DIR, "data")
LOG_DIR    = os.path.join(BASE_DIR, "logs")
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(DRIVE_BASE, exist_ok=True)

RESULTS_DIR = os.path.join(DRIVE_BASE, "Results_May_2025", "results_sac_walkforward")
SAVE_DIR    = os.path.join(RESULTS_DIR, "models_sac")
for p in [RESULTS_DIR, SAVE_DIR]:
    os.makedirs(p, exist_ok=True)

# Canonical CSV (in Results folder) + Drive mirror + local mirror
FEATURE_CSV_RESULTS = os.path.join(RESULTS_DIR, "multi_stock_feature_engineered_dataset.csv")
FEATURE_CSV_DRIVE   = os.path.join(DRIVE_BASE, "trading_data", "multi_stock_feature_engineered_dataset.csv")
FEATURE_CSV_LOCAL   = "multi_stock_feature_engineered_dataset.csv"
os.makedirs(os.path.dirname(FEATURE_CSV_DRIVE), exist_ok=True)

# ========= Feature Toggles / Data Params =========
USE_SENTIMENT  = False
USE_REGIME     = True
USE_WAVELET    = True
USE_GREEKS     = True  # placeholder (unused here)

# Action → exposure hygiene
DEAD_BAND        = 0.05   # |a| < DEAD_BAND -> 0 exposure (reduces micro-churn)
MIN_TRADE_DELTA  = 0.02   # ignore exposure changes smaller than this

# Risk controls (Phase 3)
STOP_LOSS_PCT    = 0.03   # 3% adverse move from entry triggers forced flat
TAKE_PROFIT_PCT  = 0.06   # 6% favorable move from entry triggers forced flat
COOLDOWN_STEPS   = 5      # steps to remain flat after SL/TP or attempted flip


# ========= Device & Policy (unchanged) =========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
policy_kwargs = dict(net_arch=[64, 64])

INTERVAL    = "1h"
PERIOD_DAYS = 720

# ========= Universe =========
TICKERS = [
    'AAPL','TSLA','MSFT','GOOGL','AMZN','NVDA','META','BRK-B','JPM','JNJ',
    'XOM','V','PG','UNH','MA','HD','LLY','MRK','PEP','KO',
    'BAC','ABBV','AVGO','PFE','COST','CSCO','TMO','ABT','ACN','WMT',
    'MCD','ADBE','DHR','CRM','NKE','INTC','QCOM','NEE','AMD','TXN',
    'AMGN','UPS','LIN','PM','UNP','BMY','LOW','RTX','CVX','IBM',
    'GE','SBUX','ORCL'
]
SYMBOLS = TICKERS[:]  # can override for short tests

fast_names = {
    "TSLA","NVDA","AMD","AVGO","AAPL","MSFT","AMZN","GOOGL","META","ADBE","CRM",
    "INTC","QCOM","TXN","ORCL","NEE","GE","XOM","CVX","LLY","NKE","SBUX"
}
slow_names = {
    "BRK-B","JPM","BAC","JNJ","UNH","MRK","PFE","ABBV","ABT","AMGN","PG","PEP","KO",
    "V","MA","WMT","MCD","TMO","DHR","ACN","IBM","LIN","PM","RTX","UPS","UNP","COST","HD","LOW"
}

def pick_bucket(symbol: str) -> str:
    if symbol in fast_names: return "FAST"
    if symbol in slow_names: return "SLOW"
    return "MED"

# ========= Helpers: schema + datetime =========
def _force_datetime_column(df: pd.DataFrame) -> pd.DataFrame:
    """Ensure tz-naive 'Datetime' exists, dedup and sort."""
    if isinstance(df.index, pd.DatetimeIndex):
        try:
            df.index = df.index.tz_convert(None)
        except Exception:
            try:
                df.index = df.index.tz_localize(None)
            except Exception:
                pass
        df.index.name = 'Datetime'
        df = df.reset_index()
    else:
        df = df.reset_index()
        first = df.columns[0]
        if np.issubdtype(df[first].dtype, np.datetime64):
            df = df.rename(columns={first: 'Datetime'})
        elif 'Date' in df.columns:
            df['Datetime'] = pd.to_datetime(df['Date'])
        elif 'Datetime' not in df.columns:
            df['Datetime'] = pd.to_datetime(df[first], errors='coerce')
    if 'Datetime' not in df.columns:
        raise KeyError("Failed to construct 'Datetime' from data.")
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    return df.drop_duplicates(subset=['Datetime']).sort_values('Datetime').reset_index(drop=True)

def _normalize_ohlcv(df_in: pd.DataFrame, ticker: str) -> pd.DataFrame:
    """Flatten MultiIndex, strip ticker prefixes/suffixes, map to canonical OHLCV names."""
    df = df_in.copy()
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(p) for p in col if p]) for col in df.columns]
    df.columns = [re.sub(r"\s+", " ", str(c)).strip() for c in df.columns]
    tkr = ticker.upper().replace("-", "[- ]?")
    cleaned = {}
    for c in df.columns:
        cu = c.upper()
        cu = re.sub(rf"^(?:{tkr})[\s/_-]+", "", cu)
        cu = re.sub(rf"[\s/_-]+(?:{tkr})$", "", cu)
        cleaned[c] = cu.title()
    if any(cleaned[c] != c for c in df.columns):
        df = df.rename(columns=cleaned)
    cols_ci = {c.lower(): c for c in df.columns}
    wants = {
        "Open":      ["open"],
        "High":      ["high"],
        "Low":       ["low"],
        "Close":     ["close","last","close*"],
        "Adj Close": ["adj close","adj_close","adjclose","adjusted close"],
        "Volume":    ["volume","vol"]
    }
    rename_map = {}
    for desired, alts in wants.items():
        if desired.lower() in cols_ci:
            rename_map[cols_ci[desired.lower()]] = desired
        else:
            for a in alts:
                if a in cols_ci:
                    rename_map[cols_ci[a]] = desired
                    break
    if rename_map:
        df = df.rename(columns=rename_map)
    return df

# ========= Downloader (retry + history() fallback) =========
def download_stock_data(ticker, interval="1h", period_days=720, max_retries=5, sleep_base=3):
    """
    Robust yfinance intraday downloader.
    Guarantees: Open, High, Low, Close, Volume (+Adj Close), Datetime, Symbol.
    """
    period_str = f"{int(period_days)}d"

    def _post(df: pd.DataFrame) -> pd.DataFrame:
        df = _normalize_ohlcv(df, ticker)
        df = _force_datetime_column(df)
        needed = {'Open', 'High', 'Low', 'Close', 'Volume'}
        missing = needed - set(df.columns)
        if missing:
            raise ValueError(f"Missing OHLCV columns after normalize: {missing}")
        if 'Adj Close' not in df.columns:
            df['Adj Close'] = df['Close']
        return df

    for attempt in range(1, max_retries + 1):
        try:
            log.info(f"[{ticker}] Attempt {attempt}: download({period_str}, {interval})")
            df = yf.download(
                tickers=ticker, period=period_str, interval=interval,
                progress=False, auto_adjust=False, group_by='column',
                threads=False, prepost=False, repair=True
            )
            if df is None or df.empty:
                raise ValueError("Empty data from download()")
            df = _post(df); df['Symbol'] = ticker
            log.info(f"[{ticker}] rows={len(df)} {df['Datetime'].min()}→{df['Datetime'].max()}")
            return df
        except Exception as e1:
            log.warning(f"[{ticker}] download error: {e1} | trying history()")
            try:
                hist = yf.Ticker(ticker).history(
                    period=period_str, interval=interval,
                    auto_adjust=False, actions=False
                )
                if hist is None or hist.empty:
                    raise ValueError("Empty data from history()")
                df = _post(hist); df['Symbol'] = ticker
                log.info(f"[{ticker}] (fallback) rows={len(df)} {df['Datetime'].min()}→{df['Datetime'].max()}")
                return df
            except Exception as e2:
                wait = sleep_base * attempt
                log.warning(f"[{ticker}] history() error: {e2} | retrying in {wait}s")
                time.sleep(wait)

    log.error(f"[{ticker}] Failed after {max_retries} attempts.")
    return None

# ========= Feature Engineering =========
def denoise_wavelet(series, wavelet='db1', level=2):
    s = pd.Series(series).astype(float).ffill().bfill().to_numpy()
    try:
        coeffs = pywt.wavedec(s, wavelet, mode='symmetric', level=level)
        for i in range(1, len(coeffs)):
            coeffs[i] = np.zeros_like(coeffs[i])
        rec = pywt.waverec(coeffs, wavelet, mode='symmetric')
        return pd.Series(rec[:len(s)], index=series.index)
    except Exception as e:
        log.warning(f"Wavelet denoise failed ({e}); returning raw series.")
        return pd.Series(s, index=series.index)

def add_regime(df: pd.DataFrame) -> pd.DataFrame:
    df['Vol20'] = df['Close'].pct_change().rolling(20).std()
    df['Ret20'] = df['Close'].pct_change(20)
    vol_hi   = (df['Vol20'] > df['Vol20'].median()).astype(int)
    trend_hi = (df['Ret20'].abs() > df['Ret20'].abs().median()).astype(int)
    df['Regime4'] = vol_hi * 2 + trend_hi  # 0..3
    return df

def compute_enhanced_features(df: pd.DataFrame) -> tuple[pd.DataFrame, list]:
    """
    Returns (feature_df, FEATURES).
    Input must contain: Datetime, Symbol, Open, High, Low, Close, Volume.
    """
    df = df.copy()
    req = {"Open", "High", "Low", "Close", "Volume"}
    assert req.issubset(df.columns), f"OHLCV columns missing: {req - set(df.columns)}"

    close = df["Close"].astype("float64")
    high  = df["High"].astype("float64")
    low   = df["Low"].astype("float64")
    open_ = df["Open"].astype("float64")
    vol   = (df["Volume"].astype("float64") + 1.0)

    # Returns & lags
    df["ret_1"]     = close.pct_change(1)
    df["ret_3"]     = close.pct_change(3)
    df["ret_5"]     = close.pct_change(5)
    df["ret_10"]    = close.pct_change(10)
    df["logret_1"]  = np.log(close).diff(1)

    # MAs / vol
    df["ma_5"]      = close.rolling(5).mean()
    df["ma_10"]     = close.rolling(10).mean()
    df["ma_20"]     = close.rolling(20).mean()
    df["ema_10"]    = close.ewm(span=10, adjust=False).mean()
    df["ema_20"]    = close.ewm(span=20, adjust=False).mean()
    df["std_10"]    = close.rolling(10).std()
    df["std_20"]    = close.rolling(20).std()

    df["ema10_ratio"] = df["ema_10"] / close - 1.0
    df["ema20_ratio"] = df["ema_20"] / close - 1.0

    df["z_close_20"] = (close - df["ma_20"]) / df["std_20"].replace(0, np.nan)

    # RSI(14)
    delta      = close.diff()
    up         = delta.clip(lower=0.0)
    down       = -delta.clip(upper=0.0)
    roll_up    = up.ewm(alpha=1/14, adjust=False).mean()
    roll_down  = down.ewm(alpha=1/14, adjust=False).mean()
    rs         = roll_up / roll_down.replace(0, np.nan)
    df["rsi_14"] = 100 - (100 / (1 + rs))

    # Stoch(14,3)
    ll14       = low.rolling(14).min()
    hh14       = high.rolling(14).max()
    den_14     = (hh14 - ll14).replace(0, np.nan)
    df["stoch_k"] = 100 * (close - ll14) / den_14
    df["stoch_d"] = df["stoch_k"].rolling(3).mean()

    # MACD
    ema12         = close.ewm(span=12, adjust=False).mean()
    ema26         = close.ewm(span=26, adjust=False).mean()
    macd          = ema12 - ema26
    macd_sig      = macd.ewm(span=9, adjust=False).mean()
    df["macd"]        = macd
    df["macd_signal"] = macd_sig
    df["macd_hist"]   = macd - macd_sig

    # Bollinger(20,2)
    bb_mid      = df["ma_20"]
    bb_std      = df["std_20"]
    bb_up       = bb_mid + 2 * bb_std
    bb_lo       = bb_mid - 2 * bb_std
    band_width  = (bb_up - bb_lo)
    df["bb_perc_b"]   = (close - bb_lo) / band_width.replace(0, np.nan)
    df["bb_bandwidth"] = band_width / bb_mid.replace(0, np.nan)

    # ATR(14)
    prev_close  = close.shift(1)
    tr = np.maximum(high - low,
                    np.maximum((high - prev_close).abs(), (low - prev_close).abs()))
    df["atr_14"] = tr.rolling(14).mean()

    # Volume features
    df["vol_ma_20"]    = vol.rolling(20).mean()
    df["vol_std_20"]   = vol.rolling(20).std()
    df["vol_z_20"]     = (vol - df["vol_ma_20"]) / df["vol_std_20"].replace(0, np.nan)
    df["vol_change_1"] = vol.pct_change(1)

    # Spreads
    df["hl_spread"] = (high - low) / close.replace(0, np.nan)
    df["oc_spread"] = (close - open_) / open_.replace(0, np.nan)

    # Simple cross
    df["ema10_gt_ema20"] = (df["ema_10"] > df["ema_20"]).astype("float32")

    if USE_WAVELET:
        df["denoised_close"] = denoise_wavelet(pd.Series(close, index=df.index))
    if USE_REGIME:
        df = add_regime(df)  # adds 'Regime4'

    # Clean + dtypes
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(0.0, inplace=True)

    FEATURES = [
        "ret_1","ret_3","ret_5","ret_10","logret_1",
        "ma_5","ma_10","ma_20","ema_10","ema_20","ema10_ratio","ema20_ratio",
        "std_10","std_20","z_close_20",
        "rsi_14","stoch_k","stoch_d",
        "macd","macd_signal","macd_hist",
        "bb_perc_b","bb_bandwidth",
        "atr_14",
        "vol_ma_20","vol_std_20","vol_z_20","vol_change_1",
        "hl_spread","oc_spread","ema10_gt_ema20",
    ]
    df[FEATURES] = df[FEATURES].astype("float32")
    return df, FEATURES

# ========= Orchestrator (download + features + save) =========
def build_features_if_needed() -> pd.DataFrame:
    """
    If any features CSV exists, load and return it.
    Else: download, engineer, save to RESULTS/Drive/local, and return.
    """
    for candidate in [FEATURE_CSV_LOCAL, FEATURE_CSV_RESULTS, FEATURE_CSV_DRIVE]:
        if os.path.exists(candidate):
            log.info(f"Using existing features CSV: {candidate}")
            d = pd.read_csv(candidate)
            d["Datetime"] = pd.to_datetime(d["Datetime"])
            return d

    all_dfs = []
    for i, ticker in enumerate(SYMBOLS, 1):
        log.info(f"[{i}/{len(SYMBOLS)}] {ticker} — downloading")
        raw = download_stock_data(ticker, interval=INTERVAL, period_days=PERIOD_DAYS)
        if raw is None or raw.empty:
            log.warning(f"[{ticker}] no data; skipping.")
            continue
        try:
            feats, _ = compute_enhanced_features(raw)
            if feats is not None and not feats.empty:
                all_dfs.append(feats)
                out_parq = os.path.join(DATA_DIR, f"{ticker}.parquet")
                feats.to_parquet(out_parq, index=False)
                log.info(f"[{ticker}] features={len(feats)} rows → {out_parq}")
            else:
                log.warning(f"[{ticker}] empty features; skipped.")
        except Exception as e:
            log.error(f"[{ticker}] FE failed: {e}")
        finally:
            del raw
            try: del feats
            except: pass
            gc.collect()
            time.sleep(0.2)

    if not all_dfs:
        raise RuntimeError("No usable data found for any ticker.")

    full = pd.concat(all_dfs, ignore_index=True)

    # Save to all mirrors
    full.to_csv(FEATURE_CSV_RESULTS, index=False)
    full.to_csv(FEATURE_CSV_DRIVE,   index=False)
    full.to_csv(FEATURE_CSV_LOCAL,   index=False)

    # Verify artifacts
    def _probe_artifact(path):
        ok = os.path.exists(path)
        log.info("Artifact: %s | exists=%s", path, ok)
        if not ok: return
        sz = os.path.getsize(path)
        tm = time.ctime(os.path.getmtime(path))
        try:
            head = pd.read_csv(path, nrows=2)
            cols = list(head.columns)
            sample = head.to_string(index=False)
        except Exception as e:
            cols, sample = [], f"<read error: {e}>"
        log.info("  size=%s bytes | modified=%s | cols=%s", f"{sz:,}", tm, cols[:10])
        log.info("  head:\n%s", sample)

    for pth in [FEATURE_CSV_RESULTS, FEATURE_CSV_DRIVE, FEATURE_CSV_LOCAL]:
        _probe_artifact(pth)

    log.info("Saved combined CSV → %s (rows=%d)", FEATURE_CSV_RESULTS, len(full))
    return full

# ========= SAC side toggles/knobs =========
ENABLE_PLOTS   = False
ENABLE_SLO     = True
LIVE_MODE      = False
SIM_LATENCY_MS = 0
BROKER         = "log"
USE_REGIME_TRAIN = False
USE_SENTIMENT_TRAIN = False
ENABLE_WAVELET_TRAIN = False

MAX_EXPOSURE    = 1.0
COMMISSION_BPS  = 1.0
SLIPPAGE_BPS    = 2.0
INITIAL_CAPITAL = 100_000.0

CANDIDATE_WS   = [8, 10, 12, 16, 24]
TOP_N_WINDOWS  = 1 if TEST_MODE else 3
PROBE_STEPS    = 300 if TEST_MODE else 1500
MAX_PROBE_EVAL = 400

LOG_TRADES_TRAIN  = False
LOG_TRADES_EVALCB = False
LOG_TRADES_FINAL  = True

FAST = dict(
    learning_rate=3e-4, batch_size=256, train_freq=1, gradient_steps=1,
    gamma=0.99, tau=0.02, ent_coef="auto", buffer_size=int(1e6), learning_starts=10_000,
)
SLOW = dict(
    learning_rate=1e-4, batch_size=512, train_freq=4, gradient_steps=2,
    gamma=0.995, tau=0.01, ent_coef="auto", buffer_size=int(2e6), learning_starts=20_000,
)

if FAST_TEST:
    CANDIDATE_WS   = [10]
    TOP_N_WINDOWS  = 1
    PROBE_STEPS    = 100
    MAX_PROBE_EVAL = 150
    MIN_TRAIN_STEPS   = 1000
    USE_EVAL_CALLBACK = False
    FAST["buffer_size"] = 50_000; FAST["batch_size"] = 128
    SLOW["buffer_size"] = 50_000; SLOW["batch_size"] = 128
    LOG_TRADES_FINAL = False

def get_sac_cfg(symbol: str) -> dict:
    if symbol in fast_names:
        bucket = "FAST"; base = FAST
    elif symbol in slow_names:
        bucket = "SLOW"; base = SLOW
    else:
        bucket = "DEFAULT"; base = FAST
    log.info("Config bucket for %s -> %s", symbol, bucket)
    return base | {"_bucket": bucket}

# ========= DIAGNOSTICS: start download/feature build + file proofs =========
log.info("yfinance version: %s", getattr(yf, "__version__", "unknown"))
log.info("pandas version: %s", pd.__version__)

# Force creation (writes RESULTS/DRIVE/LOCAL)
_ = build_features_if_needed()

# Resolve the dataset path to read from
CANDIDATES_DATA = [FEATURE_CSV_LOCAL, FEATURE_CSV_RESULTS, FEATURE_CSV_DRIVE]
DATA_PATH = next((p for p in CANDIDATES_DATA if os.path.exists(p)), None)
if DATA_PATH is None:
    raise FileNotFoundError("multi_stock_feature_engineered_dataset.csv not found after build.")
log.info("Resolved DATA_PATH: %s", DATA_PATH)

def _probe(path, nrows=5):
    exists = os.path.exists(path)
    log.info("Checking: %s | exists=%s", path, exists)
    if not exists: return
    sz = os.path.getsize(path); tm = time.ctime(os.path.getmtime(path))
    log.info("  size: %s bytes | modified: %s", f"{sz:,}", tm)
    try:
        head = pd.read_csv(path, nrows=nrows)
        log.info("  columns (first 10): %s ...", list(head.columns)[:10])
        log.info("\n%s", head.head(2).to_string(index=False))
    except Exception as e:
        log.warning("  peek failed: %s", e)

_probe(DATA_PATH)
_probe(FEATURE_CSV_RESULTS)
_probe(FEATURE_CSV_DRIVE)
_probe(FEATURE_CSV_LOCAL)

# Write Excel mirrors (optional)
try:
    df_check = pd.read_csv(DATA_PATH)
    local_xlsx = "multi_stock_feature_engineered_dataset.xlsx"
    drive_dir  = os.path.join(DRIVE_BASE, "trading_data")
    os.makedirs(drive_dir, exist_ok=True)
    drive_xlsx = os.path.join(drive_dir, "multi_stock_feature_engineered_dataset.xlsx")
    df_check.to_excel(local_xlsx, index=False)
    df_check.to_excel(drive_xlsx, index=False)
    log.info("Saved Excel mirrors:\n- %s\n- %s", local_xlsx, drive_xlsx)
except Exception as e:
    log.warning("Excel export skipped: %s", e)

# Optional high-level summary
try:
    df_diag = pd.read_csv(DATA_PATH)
    if "Datetime" in df_diag.columns:
        df_diag["Datetime"] = pd.to_datetime(df_diag["Datetime"], errors="coerce")
        dmin, dmax = df_diag["Datetime"].min(), df_diag["Datetime"].max()
        log.info("Combined dataset shape: %s", df_diag.shape)
        log.info("Range: %s → %s", dmin, dmax)
        if "Symbol" in df_diag.columns:
            counts = df_diag["Symbol"].value_counts().sort_index()
            log.info("Per-ticker counts:\n%s", counts.to_string())
        if df_diag.isna().any().any():
            na_cols = df_diag.columns[df_diag.isna().any()].tolist()
            log.warning("NaNs detected in columns: %s", na_cols)
        else:
            log.info("No NaNs detected.")
except Exception as e:
    log.warning("Summary skipped: %s", e)
# ========= DIAGNOSTICS: end =========

# ========= Resume: detect already-trained symbols from SAVE_DIR =========
os.makedirs(SAVE_DIR, exist_ok=True)
pat = re.compile(r"^sac_([A-Za-z0-9\-]+)(?:_.*)?\.zip$", re.IGNORECASE)

df = pd.read_csv(DATA_PATH)
if "Datetime" in df.columns:
    df["Datetime"] = pd.to_datetime(df["Datetime"], utc=True)

symbols_all = sorted({str(s).upper() for s in df["Symbol"].unique()})
completed_symbols = sorted({
    pat.match(f).group(1).upper()
    for f in os.listdir(SAVE_DIR) if pat.match(f)
})
remaining_symbols = [s for s in symbols_all if s not in completed_symbols]

log.info("Completed: %d symbols | Remaining: %d symbols", len(completed_symbols), len(remaining_symbols))
log.info("Completed (first 10): %s", completed_symbols[:10])
log.info("Remaining (first 10): %s", remaining_symbols[:10])

# Ensure subdirs
for sub in ["scalers", "features", "signals", "plots", "vecnorms", "tmp", "model_info"]:
    os.makedirs(os.path.join(RESULTS_DIR, sub), exist_ok=True)

# Run-level inference config
inference_cfg_path = os.path.join(RESULTS_DIR, "inference_config.json")
if not os.path.exists(inference_cfg_path):
    run_infer_cfg = {
        "algo": "SAC",
        "action_mode": "deterministic",
        "conf_source": "none",
        "conf_thresh": None,
        "n_eval_runs": 1,
        "created_at": datetime.now().isoformat(),
    }
    with open(inference_cfg_path, "w") as f:
        json.dump(run_infer_cfg, f, indent=2)
    log.info("Wrote inference_config.json -> %s", inference_cfg_path)

# Optional live helper
def latest_df_for_symbol(symbol: str, horizon_days: int = 5, interval: str = "1m"):
    end = datetime.now(timezone.utc)
    start = end - timedelta(days=horizon_days)
    df_live = yf.download(
        symbol, start=start.strftime("%Y-%m-%d"), end=end.strftime("%Y-%m-%d"),
        interval=interval, progress=False, auto_adjust=False,
    )
    if df_live is None or df_live.empty:
        log.warning("latest_df_for_symbol: no data for %s", symbol); return None
    df_live = df_live.reset_index()
    if "Datetime" not in df_live.columns:
        ts_col = df_live.columns[0]
        df_live = df_live.rename(columns={ts_col: "Datetime"})
    df_live["Datetime"] = pd.to_datetime(df_live["Datetime"], utc=True)
    df_live["Symbol"] = symbol
    try:
        feats, _ = compute_enhanced_features(df_live)
        return feats
    except Exception as e:
        log.exception("Feature engineering failed for live %s: %s", symbol, e)
        return None

log.info(
    "Toggles | TEST_MODE=%s ENABLE_PLOTS=%s ENABLE_SLO=%s LIVE_MODE=%s SIM_LATENCY_MS=%s "
    "BROKER=%s USE_REGIME_TRAIN=%s USE_SENTIMENT_TRAIN=%s ENABLE_WAVELET_TRAIN=%s",
    TEST_MODE, ENABLE_PLOTS, ENABLE_SLO, LIVE_MODE, SIM_LATENCY_MS,
    BROKER, USE_REGIME_TRAIN, USE_SENTIMENT_TRAIN, ENABLE_WAVELET_TRAIN
)

# If build function returns a DataFrame, prefer that:
try:
    df = build_features_if_needed()
    if isinstance(df, str):  # not expected, but guard
        df = pd.read_csv(df)
except Exception:
    df = pd.read_csv(DATA_PATH)
df["Datetime"] = pd.to_datetime(df["Datetime"], utc=True)

# ========= Time windows =========
max_ts = df["Datetime"].max()
test_end_dt    = max_ts
train_start_dt = test_end_dt - timedelta(days=729)
train_end_dt   = train_start_dt + timedelta(days=365)
test_start_dt  = train_end_dt

train_start = train_start_dt.strftime("%Y-%m-%d")
train_end   = train_end_dt.strftime("%Y-%m-%d")
test_start  = test_start_dt.strftime("%Y-%m-%d")
test_end    = test_end_dt.strftime("%Y-%m-%d")

train_start_ts = pd.to_datetime(train_start, utc=True)
train_end_ts   = pd.to_datetime(train_end,   utc=True)
test_start_ts  = pd.to_datetime(test_start,  utc=True)
test_end_ts    = pd.to_datetime(test_end,    utc=True)

log.info("Bounds UTC | train=[%s, %s) test=[%s, %s]",
         train_start_ts, train_end_ts, test_start_ts, test_end_ts)

# ========= Choose run_symbols =========
run_symbols = remaining_symbols
if TEST_MODE:
    fast_avail = [s for s in remaining_symbols if s in fast_names]
    slow_avail = [s for s in remaining_symbols if s in slow_names]
    pick_fast  = fast_avail[0] if fast_avail else (remaining_symbols[0] if remaining_symbols else None)
    pick_slow  = next((s for s in slow_avail if s != pick_fast), None) if slow_avail else None
    run_symbols = [s for s in [pick_fast, pick_slow] if s is not None]
    log.info("TEST_MODE: limiting run to (FAST, SLOW) = %s", run_symbols)
if FAST_TEST:
    run_symbols = run_symbols[:1]
    log.info("FAST_TEST: limiting to one symbol -> %s", run_symbols)

# ========= Device & Policy =========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
policy_kwargs = dict(net_arch=[64, 64])

class ContinuousTradingEnv(gym.Env):
    """
    Continuous exposure env with costs + risk controls.

      Phase 2:
        - Action a in [-1,1] -> exposure in [-MAX_EXPOSURE, MAX_EXPOSURE]
        - Deadband around zero, min trade delta
        - Commission + slippage (bps) on position change notional

      Phase 3:
        - Stop-loss / Take-profit based on entry price
        - Cooldown after forced exits or attempted flip (optional)
    """
    metadata = {"render_modes": []}

    def __init__(
        self, df: pd.DataFrame, frame_bound: tuple, window_size: int,
        max_exposure: float = MAX_EXPOSURE,
        commission_bps: float = COMMISSION_BPS,
        slippage_bps: float = SLIPPAGE_BPS,
        initial_capital: float = INITIAL_CAPITAL,
        log_trades: bool = True,
        # Phase 2/3 params (safe defaults keep old behavior)
        dead_band: float = 0.0,
        min_trade_delta: float = 0.0,
        stop_loss_pct: float | None = None,
        take_profit_pct: float | None = None,
        cooldown_steps: int = 0,
    ):
        super().__init__()
        assert "Close" in df.columns and "Datetime" in df.columns, "DataFrame must have Close & Datetime"
        self.raw_df = df.sort_values("Datetime").reset_index(drop=True).copy()
        self.window_size = int(window_size)
        self.start_tick, self.end_tick = int(frame_bound[0]), int(frame_bound[1])
        assert self.end_tick <= len(self.raw_df), "frame_bound[1] exceeds data length"
        assert self.start_tick >= self.window_size, "start tick must be >= window_size"

        self.prices = self.raw_df["Close"].values.astype(np.float64)
        feats = self.raw_df.drop(columns=["Symbol", "Datetime", "Close"], errors="ignore")
        feats = feats.select_dtypes(include=[np.number]).astype(np.float32).values
        self.features = feats
        n_feat = self.features.shape[1]

        self.action_space = Box(low=-1.0, high=1.0, shape=(1,), dtype=np.float32)
        self.observation_space = Box(low=-np.inf, high=np.inf, shape=(self.window_size, n_feat), dtype=np.float32)

        # Costs & exposure
        self.max_exposure = float(max_exposure)
        self.bps_total = (float(commission_bps) + float(slippage_bps)) / 10_000.0
        self.initial_capital = float(initial_capital)

        # Phase 2/3 settings
        self.dead_band = float(dead_band)
        self.min_trade_delta = float(min_trade_delta)
        self.stop_loss_pct = float(stop_loss_pct) if stop_loss_pct is not None else None
        self.take_profit_pct = float(take_profit_pct) if take_profit_pct is not None else None
        self.cooldown_steps = int(cooldown_steps)

        self.log_trades = bool(log_trades)
        self.log_every = 500 if TEST_MODE else 2000

        # Runtime state
        self.current_tick = None
        self.position = None
        self.portfolio_value = None
        self.entry_price = None            # for SL/TP
        self.cooldown_until = -1           # tick index until which entries are blocked
        self.done_tick = self.end_tick - 1

    def _obs(self):
        start = self.current_tick - self.window_size
        obs = self.features[start:self.current_tick]
        return obs.astype(np.float32)

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        self.current_tick = max(self.start_tick, self.window_size)
        self.position = 0.0
        self.portfolio_value = self.initial_capital
        self.entry_price = None
        self.cooldown_until = -1
        obs = self._obs()
        info = {
            "portfolio_value": float(self.portfolio_value),
            "position": float(self.position),
            "price": float(self.prices[self.current_tick]),
            "trade_cost": 0.0,
            "risk_event": None,
            "cooldown_left": 0,
            "entry_price": None,
        }
        return obs, info

    # ---- helpers (Phase 3) ----
    def _pnl_from_entry(self, price_now: float) -> float:
        """Signed P&L % vs entry, positive if in the trade's favor."""
        if self.entry_price is None or self.entry_price <= 0 or self.position == 0.0:
            return 0.0
        side = 1.0 if self.position > 0 else -1.0
        return side * (price_now / self.entry_price - 1.0)

    def _apply_risk_controls(self, price_t: float, proposed_pos: float):
        """
        Returns (final_target_pos, event, cooldown_left).
        - Blocks sign flips by forcing flat and starting cooldown
        - Applies SL/TP by forcing flat and starting cooldown
        - Enforces cooldown (no new entries while cooling down)
        """
        event = None
        cooldown_left = max(0, self.cooldown_until - self.current_tick)

        # If cooling down already, stay/force flat until cooldown ends
        if self.position == 0.0 and self.current_tick < self.cooldown_until:
            return 0.0, "cooldown", cooldown_left

        # Sign-flip guard: from long->short or short->long in one step
        if (
            self.cooldown_steps > 0 and
            self.position != 0.0 and
            np.sign(proposed_pos) != np.sign(self.position) and
            abs(proposed_pos) >= self.min_trade_delta
        ):
            self.cooldown_until = self.current_tick + self.cooldown_steps
            return 0.0, "flip_block", self.cooldown_steps

        # SL / TP checks (only while in a position)
        if self.position != 0.0:
            pnl_entry = self._pnl_from_entry(price_t)
            if (self.stop_loss_pct is not None) and (pnl_entry <= -self.stop_loss_pct):
                self.cooldown_until = self.current_tick + self.cooldown_steps
                return 0.0, "stop_loss", self.cooldown_steps
            if (self.take_profit_pct is not None) and (pnl_entry >= self.take_profit_pct):
                self.cooldown_until = self.current_tick + self.cooldown_steps
                return 0.0, "take_profit", self.cooldown_steps

        # No intervention
        return proposed_pos, event, cooldown_left

    def step(self, action):
        # ----- proposed exposure from action (Phase 2) -----
        a = float(np.array(action).reshape(-1)[0])
        if abs(a) < self.dead_band:
            a = 0.0
        target_pos = float(np.clip(a, -1.0, 1.0)) * self.max_exposure
        target_pos = float(np.clip(target_pos, -self.max_exposure, self.max_exposure))

        # ignore tiny nudges
        if abs(target_pos - (0.0 if self.position is None else self.position)) < self.min_trade_delta:
            target_pos = self.position

        price_t   = float(self.prices[self.current_tick])
        old_pos   = float(0.0 if self.position is None else self.position)

        # ----- Phase 3: risk controls override -----
        target_pos, risk_event, cooldown_left = self._apply_risk_controls(price_t, target_pos)

        # ----- transaction cost on position change -----
        base_cash = self.portfolio_value if self.portfolio_value else self.initial_capital
        trade_notional = abs(target_pos - old_pos) * base_cash
        trade_cost = self.bps_total * trade_notional

        # apply trade & costs
        self.position = float(target_pos)
        self.portfolio_value = float(base_cash - trade_cost)
        if not np.isfinite(self.portfolio_value) or self.portfolio_value <= 0.0:
            self.portfolio_value = max(1e-8, float(self.portfolio_value))

        # ----- step return -----
        next_tick = self.current_tick + 1
        terminated = next_tick >= len(self.prices) or self.current_tick >= self.done_tick
        price_tp1 = float(self.prices[min(next_tick, len(self.prices) - 1)])
        ret = 0.0 if price_t <= 0 else (price_tp1 / price_t - 1.0)

        v_prev = self.portfolio_value
        self.portfolio_value = v_prev * (1.0 + self.position * ret)

        # reward is post-cost step return (clipped)
        reward = 0.0 if v_prev <= 0 else (self.portfolio_value - v_prev) / v_prev
        reward = float(np.clip(reward, -1.0, 1.0))

        # ----- entry/exit bookkeeping -----
        if old_pos == 0.0 and self.position != 0.0:
            self.entry_price = price_t
        if self.position == 0.0:
            self.entry_price = None

        # optional logs
        if self.log_trades and (abs(target_pos - old_pos) > 1e-9 or risk_event):
            log.info(
                "t=%d | px=%.4f | pos %.3f→%.3f | notional=%.2f | cost=%.2f bps=%.2f | risk=%s | cd=%d | V=%.2f",
                self.current_tick, price_t, old_pos, self.position,
                trade_notional, trade_cost, self.bps_total * 10_000.0,
                (risk_event or "none"), int(cooldown_left), self.portfolio_value
            )

        # advance
        self.current_tick = next_tick
        obs = self._obs()
        info = {
            "portfolio_value": float(self.portfolio_value),
            "position": float(self.position),
            "price": float(price_tp1),
            "trade_cost": float(trade_cost),
            "risk_event": risk_event,
            "cooldown_left": int(max(0, self.cooldown_until - self.current_tick)) if self.cooldown_until >= 0 else 0,
            "entry_price": None if self.entry_price is None else float(self.entry_price),
        }
        truncated = False
        return obs.astype(np.float32), reward, bool(terminated), bool(truncated), info


# ========= factories (pass Phase 2/3 params) =========
def _risk_kwargs():
    return dict(
        dead_band=DEAD_BAND,
        min_trade_delta=MIN_TRADE_DELTA,
        stop_loss_pct=(STOP_LOSS_PCT if ENABLE_SLO else None),
        take_profit_pct=(TAKE_PROFIT_PCT if ENABLE_SLO else None),
        cooldown_steps=(COOLDOWN_STEPS if ENABLE_SLO else 0),
    )

def make_train_env():
    return ContinuousTradingEnv(
        train_df, frame_bound=(WS_USE, len(train_df)), window_size=WS_USE,
        max_exposure=MAX_EXPOSURE, commission_bps=COMMISSION_BPS,
        slippage_bps=SLIPPAGE_BPS, initial_capital=INITIAL_CAPITAL,
        log_trades=LOG_TRADES_TRAIN, **_risk_kwargs()
    )

def make_test_env_for_callback():
    return ContinuousTradingEnv(
        test_df, frame_bound=(WS_USE, len(test_df)), window_size=WS_USE,
        max_exposure=MAX_EXPOSURE, commission_bps=COMMISSION_BPS,
        slippage_bps=SLIPPAGE_BPS, initial_capital=INITIAL_CAPITAL,
        log_trades=LOG_TRADES_EVALCB, **_risk_kwargs()
    )

for symbol in run_symbols:
    sac_cfg = get_sac_cfg(symbol)
    sdf = df[df["Symbol"] == symbol].sort_values("Datetime")
    train_df = sdf[(sdf["Datetime"] >= train_start_ts) & (sdf["Datetime"] <  train_end_ts)].reset_index(drop=True)
    test_df  = sdf[(sdf["Datetime"] >= test_start_ts)  & (sdf["Datetime"] <= test_end_ts)].reset_index(drop=True)
    # >>> WS sweep block goes here <<<

# ==== WS sweep (train+eval per window size, rank by Sharpe) ====

def train_eval_for_ws(ws: int):
    # --- factories bound to this ws ---
    def make_train_env():
        return ContinuousTradingEnv(
            train_df, frame_bound=(ws, len(train_df)), window_size=ws,
            max_exposure=MAX_EXPOSURE, commission_bps=COMMISSION_BPS,
            slippage_bps=SLIPPAGE_BPS,initial_capital=INITIAL_CAPITAL,
            log_trades=LOG_TRADES_TRAIN, **_risk_kwargs()
        )

    def make_test_env_for_callback():
        return ContinuousTradingEnv(
            test_df, frame_bound=(ws, len(test_df)), window_size=ws,
            max_exposure=MAX_EXPOSURE, commission_bps=COMMISSION_BPS,
            slippage_bps=SLIPPAGE_BPS, initial_capital=INITIAL_CAPITAL,
            log_trades=LOG_TRADES_EVALCB, **_risk_kwargs()
        )

    def make_test_env_for_final():
        return ContinuousTradingEnv(
            test_df, frame_bound=(ws, len(test_df)), window_size=ws,
            max_exposure=MAX_EXPOSURE, commission_bps=COMMISSION_BPS,
            slippage_bps=SLIPPAGE_BPS, initial_capital=INITIAL_CAPITAL,
            log_trades=LOG_TRADES_FINAL, **_risk_kwargs()
        )

    # --- vec wrapper(s) ---
    train_env = DummyVecEnv([make_train_env])
    train_env = VecNormalize(train_env, training=True, norm_obs=True, norm_reward=False, clip_obs=10.0)
    train_env.seed(SEED); _ = train_env.reset()

    eval_env  = DummyVecEnv([lambda: Monitor(make_test_env_for_callback())])
    eval_env  = VecNormalize(eval_env, training=False, norm_obs=True, norm_reward=False, clip_obs=10.0)
    eval_env.obs_rms = train_env.obs_rms
    eval_env.seed(SEED); _ = eval_env.reset()

    # --- SAC config (bucketed) ---
    loc_cfg = sac_cfg.copy()
    if TEST_MODE:
        loc_cfg["learning_starts"] = 0

    model = SAC(
        "MlpPolicy", train_env, device=device, policy_kwargs=policy_kwargs, verbose=0,
        learning_rate=loc_cfg["learning_rate"], batch_size=loc_cfg["batch_size"],
        train_freq=loc_cfg["train_freq"], gradient_steps=loc_cfg["gradient_steps"],
        gamma=loc_cfg["gamma"], tau=loc_cfg["tau"], ent_coef=loc_cfg["ent_coef"],
        buffer_size=loc_cfg["buffer_size"], learning_starts=loc_cfg["learning_starts"], seed=SEED,
    )

    # --- schedule (reuse your fast/full logic) ---
    if FAST_TEST:
        eval_every  = 1_000
        total_steps = max(200, min(MIN_TRAIN_STEPS, len(train_df)))
        loc_cfg["learning_starts"] = 0
        callback = None if not USE_EVAL_CALLBACK else EvalCallback(
            eval_env, best_model_save_path=os.path.join(RESULTS_DIR, "tmp", f"best_{symbol}_ws{ws}"),
            eval_freq=eval_every, n_eval_episodes=1,
            callback_after_eval=StopTrainingOnNoModelImprovement(1, 2, verbose=0), verbose=0,
        )
    else:
        eval_every  = 1_000 if TEST_MODE else 5_000
        total_steps = (min(4_000, len(train_df) * 2) if TEST_MODE else min(25_000, len(train_df) * 10))
        if TEST_MODE: loc_cfg["learning_starts"] = 0
        callback = EvalCallback(
            eval_env, best_model_save_path=os.path.join(RESULTS_DIR, "tmp", f"best_{symbol}_ws{ws}"),
            eval_freq=eval_every, n_eval_episodes=1,
            callback_after_eval=StopTrainingOnNoModelImprovement(2, 4, verbose=0), verbose=0,
        )

    model.learn(total_timesteps=total_steps, callback=callback)

    # --- save artifacts with WS suffix ---
    model_path = f"{SAVE_DIR}/sac_{symbol}_ws{ws}"
    vec_path   = f"{RESULTS_DIR}/vecnorms/{symbol}_ws{ws}_vecnorm.pkl"
    model.save(model_path)
    train_env.save(vec_path)

    # --- clean eval + rollout to get NAV for Sharpe ranking ---
    final_eval_raw = DummyVecEnv([make_test_env_for_final])
    final_eval_raw.seed(SEED)
    final_eval = VecNormalize.load(vec_path, final_eval_raw)
    final_eval.training = False
    final_eval.norm_reward = False
    final_eval.obs_rms = train_env.obs_rms
    final_eval.seed(SEED)
    obs = final_eval.reset()

    portfolio, positions = [], []
    buy_hold = float(INITIAL_CAPITAL) * (test_df["Close"].iloc[-1] / test_df["Close"].iloc[0])

    for i in range(len(test_df)):
        action, _ = model.predict(obs, deterministic=True)
        obs, _, dones, infos = final_eval.step(action)
        info0 = infos[0]
        portfolio.append(float(info0.get("portfolio_value", np.nan)))
        positions.append(float(info0.get("position", 0.0)))
        if bool(dones[0]):
            break

    # save signals for this ws
    aligned_len = min(len(test_df), len(portfolio))
    signals_path = f"{RESULTS_DIR}/signals/{symbol}_ws{ws}_sac_signals.csv"
    test_eval_df = test_df.iloc[:aligned_len].copy()
    test_eval_df["Position"] = positions[:aligned_len]
    test_eval_df["Portfolio"] = portfolio[:aligned_len]
    test_eval_df.to_csv(signals_path, index=False)

    # minimal metrics (for ranking only)
    final_value = portfolio[-1] if portfolio else INITIAL_CAPITAL
    ret_series  = pd.Series(portfolio).pct_change().dropna()
    sharpe = (ret_series.mean() / ret_series.std()) * np.sqrt(252) if ret_series.std() and ret_series.std() > 0 else 0.0

    # cleanup
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return dict(
        ws=ws,
        sharpe=float(sharpe),
        final_value=float(final_value),
        buy_hold=float(buy_hold),
        model_zip=model_path + ".zip",
        vecnorm_pkl=vec_path,
        signals_csv=signals_path
    )

# === run sweep ===
results_ws = []
top_heap = []
TOP_N_WINDOWS = 3

# If FAST_TEST trimmed CANDIDATE_WS to [10], you'll only see one; for full runs, include more WS.
for WS in CANDIDATE_WS:
    try:
        res = train_eval_for_ws(WS)
        results_ws.append({
            "Symbol": symbol,
            "WS": WS,
            "Sharpe": round(res["sharpe"], 3),
            "SAC_Portfolio": round(res["final_value"], 2),
            "BuyHold": round(res["buy_hold"], 2),
            "ModelZip": res["model_zip"],
            "VecNorm": res["vecnorm_pkl"],
            "SignalsCSV": res["signals_csv"],
        })
        # maintain top-N by Sharpe
        heapq.heappush(top_heap, (res["sharpe"], WS, res))
        if len(top_heap) > TOP_N_WINDOWS:
            heapq.heappop(top_heap)
    except Exception as e:
        log.warning(f"WS={WS} failed: {e}")



# save per-symbol WS sweep summary
ws_summary_path = os.path.join(RESULTS_DIR, f"{symbol}_sac_ws_sweep_summary.csv")
pd.DataFrame(results_ws).sort_values("Sharpe", ascending=False).to_csv(ws_summary_path, index=False)

# report winners (like PPO)
top_sorted = sorted(top_heap, key=lambda t: t[0], reverse=True)
log.info(f"Top {min(TOP_N_WINDOWS, len(top_sorted))} window sizes for {symbol}: " +
         ", ".join([f"WS={w} (Sharpe={s:.3f})" for s, w, _ in top_sorted]))

if torch.cuda.is_available():
    torch.cuda.empty_cache()